In [ ]:
# 1. Install Android Tools and Web-Viewer directly
print("📦 Installing Linux Android Bridge...")
!apt-get update -qq
!apt-get install -y adb wget curl python3-pip -qq > /dev/null
!pip install -q flask-cors

# 2. Download the Web Viewer (scrcpy-web)
print("🌐 Setting up Web Interface...")
!npm install -g ws-scrcpy > /dev/null 2>&1 &

# 3. Start a Virtual Android Instance (Emulator-less)
print("📱 Starting Android Core...")
import os, subprocess, time, threading

# This downloads a pre-compiled Android binary that doesn't need Docker
!wget -q https://github.com/remote-android/redroid-bin/releases/download/v1/redroid-bin.tar.gz
!tar -xzf redroid-bin.tar.gz
get_ipython().system_raw('./redroid-bin &')
time.sleep(10)

# 4. Launch Cloudflare (Binary Mode)
print("🔗 Generating Link...")
!wget -q -O cf https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cf

def run_tunnel():
    # Using port 7860 which ws-scrcpy uses
    os.system('./cf tunnel --url http://localhost:7860')

threading.Thread(target=run_tunnel, daemon=True).start()

print("\n" + "="*60)
print("🚀 WAIT 15 SECONDS - THE LINK WILL APPEAR BELOW")
print("="*60)
# This force-prints the link even if the logs are messy
time.sleep(15)
!grep -o 'https://[-a-zA-Z0-9.]*\.trycloudflare\.com' <(timeout 5s ./cf tunnel --url http://localhost:7860 2>&1)